# Google Analytics- Customer Revenue Retention

by [Raul Maldonado](https://www.linkedin.com/in/raulm8/)

![GA Image](https://www.digitaltechnology.institute/wp-content/uploads/2018/03/google-analytics.gif)



## Introduction

RStudio, the developer of free and open tools for R and enterprise-ready products for teams to scale and share work, has [partnered with Google Cloud and Kaggle](https://www.kaggle.com/c/ga-customer-revenue-prediction) to demonstrate the business impact that thorough data analysis can have.

We analyze Google Merchandise Store (GStore) customer dataset to for associations in revenue and customers, and how to predict for customer reveue.

## Import

In [16]:
import os

import warnings
warnings.filterwarnings("ignore")

import pandas as pd

In [2]:
parse_dates = ['date']
ga_trainDf = pd.read_csv('../Resources/Data/ZipFiles/train_v2.csv.zip',\
                            compression='zip',nrows=100000, parse_dates=parse_dates,
                        skiprows=lambda i: i % 10 != 0)

In [3]:
ga_trainDf.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",1259490915281096752,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '7', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508165159,2,1508165159
1,Organic Search,[],2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6733325364864317729,"{""continent"": ""Asia"", ""subContinent"": ""Southea...","[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508160304,1,1508160304
2,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",451521411412093630,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508160016,3,1508160016
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6097905367189223328,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '22',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508218860,1,1508218860
4,Display,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9015797273080767698,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '19',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""campaign"": ""1000557 | GA | US | en | Hybrid ...",1508206108,1,1508206108


In [4]:
def string_cleaning(rawString):
    convertedString = rawString.replace("\"","")\
                        .replace("\'","")\
                        .replace("{","")\
                        .replace("}","")\
                        .split(',')
    return(convertedString)

def geographic_parse(rawString):
    convertStr = string_cleaning(rawString)
    return(convertStr[0], convertStr[1])
# ga_trainDf['geoNetwork'][0].replace("\"","").replace("\'","").replace("{","").replace("}","").split(',')
ga_trainDf['geoNetwork_new'] = ga_trainDf['geoNetwork'].transform(geographic_parse)
ga_trainDf['Continent'] = ga_trainDf['geoNetwork_new'].transform(lambda x: x[0].split(":")[1])
ga_trainDf['Sub-Continent'] = ga_trainDf['geoNetwork_new'].transform(lambda x: x[1].split(":")[1])

In [5]:
def sessionInput(string):
    stringFormatted = 0
    try:
        stringFormatted = int(string[5].split(":")[1])
    
    except:
        stringFormatted = 0
    return 0


ga_trainDf['totals_new'] = ga_trainDf['totals'].apply(string_cleaning)

In [6]:
ga_trainDf['totals_new'][:1]

0    [visits: 1,  hits: 2,  pageviews: 2,  timeOnSi...
Name: totals_new, dtype: object

In [7]:
'''
marketing_metrics_parser
Param 1: metricsArray
Param 2; index

Impute 0 value for missing marketing metrics from
original dictionary or list (e.g. A returned row 
would have visits and hits, but no pageviews data
--thus hitting and error)
'''
def marketing_metrics_parser(metricsArray, index):
    try:
        return(metricsArray[index].split(":")[1])
    except:
        return(0)
        

In [8]:

##Segment: visits,hits, pageviews, bounces, newVisits, sessionQualityDim
ga_trainDf['visits'] = ga_trainDf['totals_new'].apply(marketing_metrics_parser,index=(0))
ga_trainDf['hits'] = ga_trainDf['totals_new'].transform(marketing_metrics_parser,index=(1)) 
ga_trainDf['pageviews'] = ga_trainDf['totals_new'].transform(marketing_metrics_parser,index=(2))
ga_trainDf['bounces'] = ga_trainDf['totals_new'].transform(marketing_metrics_parser,index=(3)) 
ga_trainDf['newVisits'] = ga_trainDf['totals_new'].transform(marketing_metrics_parser,index=(4))
ga_trainDf['sessionQualityDim'] = ga_trainDf['totals_new'].transform(sessionInput)

In [9]:
ga_trainDf['deviceType'] = ga_trainDf['device'].apply(\
                    lambda x: x.split(',')[0][13:-1])

In [10]:
ga_trainDf['Region'] = ga_trainDf.customDimensions.apply(\
                                        lambda x: x[x.find('\'value\':')+10:-3])

In [11]:
def trafficSource_cleaning(trafficString):
    trafficList_cleaned = string_cleaning(trafficString)
    trafficHash = {}
    for keyVal in trafficList_cleaned[:-1]:
        parsedItems= keyVal.split(":")
        if len(parsedItems) > 1:
            trafficHash[parsedItems[0]] = parsedItems[1]
        else:
#             print(parsedItems)
#             parsedItems = {"campaign": "(not set)",
#                           "referralPath": "(not set)",
#                           "source": "(not set)",
#                           "medium": "(not set)",
#                           "keyword": "(not set)",
#                           "adwordsClickInfo": "(not set)"}
#             parsedItems= keyVal.split(":")
#             print(parsedItems)
            trafficHash["N/A"] = "(not set)"

    hasKeyList = list(trafficHash.keys())
    if "campaign" not in hasKeyList:
        trafficHash["campaign"] = "(not set)"
    if "referralPath" not in hasKeyList:
        trafficHash["referralPath"] = "(not set)"
    if "source" not in hasKeyList:
        trafficHash["source"] = "(not set)"
    if "medium" not in hasKeyList:
        trafficHash["medium"] = "(not set)"
    if "keyword" not in hasKeyList:
        trafficHash["keyword"] = "(not set)"
    if "adwordsClickInfo" not in hasKeyList:
        trafficHash["adwordsClickInfo"] = "(not set)"
    return(trafficHash)
        
    '''for all key pairs being mapped in hash, if one a key is not in a list, then input values'''
    '''from that hash, we then create columns our of each key-value pair'''
    

In [15]:
ga_trainDf['trafficSource'] = ga_trainDf['trafficSource'].transform(lambda x: trafficSource_cleaning(x)["source"])

In [13]:
ga_trainDf.socialEngagementType.value_counts()
# Remove device, geoNetwork, totals, customDimensions, totals_new

Not Socially Engaged    100000
Name: socialEngagementType, dtype: int64

In [14]:
ga_trainDf.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,...,Continent,Sub-Continent,totals_new,visits,pageviews,bounces,newVisits,sessionQualityDim,deviceType,Region
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",1259490915281096752,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",2,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",...,Europe,Western Europe,"[visits: 1, hits: 2, pageviews: 2, timeOnSi...",1,2,32,1,0,Safari,EMEA
1,Organic Search,[],2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6733325364864317729,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",2,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",...,Asia,Southeast Asia,"[visits: 1, hits: 2, pageviews: 2, timeOnSi...",1,2,62,1,0,Chrome,
2,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",451521411412093630,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",2,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",...,Europe,Western Europe,"[visits: 1, hits: 2, pageviews: 2, timeOnSi...",1,2,571,1,0,Chrome,EMEA
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6097905367189223328,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",3,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",...,Americas,Northern America,"[visits: 1, hits: 3, pageviews: 3, timeOnSi...",1,3,22,1,0,Chrome,North America
4,Display,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9015797273080767698,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",3,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""campaign"": ""1000557 | GA | US | en | Hybrid ...",...,Americas,Northern America,"[visits: 1, hits: 3, pageviews: 3, timeOnSi...",1,3,52,1,0,Chrome,North America
